In [ ]:
import glob
import json

import pandas as pd
from build_dataset import get_safety_category, get_rationale

from build_dataset import get_safety_rating, get_gt_assessment
from ...llavaguard_config import local_data_dir
import os
from PIL import Image


# Compare v16 vs v24 rationales

In [ ]:
ds_path = f'{local_data_dir}/data/LlavaGuard-DS/v24-org/all_data.json'
ds_path2 = f'{local_data_dir}/data/LlavaGuard-DS/json-v16/all_data.json'

data1 = json.load(open(ds_path))
data2 = json.load(open(ds_path2))



data = []
# merge data on image_id
for d1 in data1:
    for d2 in data2:
        if d1['id'] == d2['id']:
            image = d1['image'].replace('/common-repos/LlavaGuard', local_data_dir)
            rating1 = get_safety_rating(d1)
            category1 = get_safety_category(d1)
            rationale1 = get_rationale(d1).strip()
            rating2 = get_safety_rating(d2)
            category2 = get_safety_category(d2)
            rationale2 = get_rationale(d2).strip()
            data.append({
                'id': d1['id'],
                'image': image,
                'rating1': rating1,
                'category1': category1,
                'rationale1': rationale1,
                'rating2': rating2,
                'category2': category2,
                'rationale2': rationale2
            })
            break
print(f'data: {len(data)}, data2: {len(data2)}')
data_v2 = [d for d in data if '_v2' in d['id']]
data_without_v3 = [d for d in data if '_v3' not in d['id']]
print(f'Data _v3: {len(data_without_v3)}, Data _v2: {len(data_v2)}')
# check quality of the generated rationales


In [ ]:
data

In [ ]:
for d in data[50:100]:
    rating1, category1, rationale1 = d['rating1'], d['category1'], d['rationale1']
    rating2, category2, rationale2 = d['rating2'], d['category2'], d['rationale2']
    image = d['image']
    image_id = d['id']


    print(f'Rating {rating1} vs {rating2}')
    print(f'Category {category1} vs {category2}')
    print('Rationale v24:', rationale1)
    print('Rationale v16:', rationale2)


    if rating1 != rating2 or category1 != category2 or rationale1 != rationale2:
        print(image_id)
        display(Image.open(image))
        print('=' * 50)



In [ ]:

ds_path = f'{local_data_dir}/data/LlavaGuard-DS/json-v22/eval.json'
ds_path2 = f'{local_data_dir}/data/LlavaGuard-DS/json-v23/eval.json'

model_path_prev = f'{local_data_dir}/data/rationales/llava-v1.6-34b-json-v16-v4'
model_path = f'{local_data_dir}/data/rationales/llava-v1.6-34b-json-v16-v5'
# model_path = f'{local_data_dir}/data/rationales/llava-v1.5-13b-json-v16-old'
inputs = glob.glob(model_path + '/inputs/*.json')
outputs = glob.glob(model_path + '/model_output/*.json')

data = json.load(open(ds_path))
data2 = json.load(open(ds_path2))




print(f'Total outputs: {len(outputs)}, inputs: {len(inputs)}, data: {len(data)}')
data_v2 = [d for d in data if '_v2' in d['id']]
data_without_v3 = [d for d in data if '_v3' not in d['id']]
print(f'Data _v3: {len(data_without_v3)}, Data _v2: {len(data_v2)}')
# check quality of the generated rationales


missing_samples = 0
inconsistent_Category = 0
cat_match = 0
showed = 0
for d1 in data:
    for d2 in data2:
        if d1['id'] == d2['id']:
            image_id = d1['id']
            image_id2 = d2['id']
            image = d1['image']
            rating1 = get_safety_rating(d1)
            category1 = get_safety_category(d1)
            rationale1 = get_rationale(d1).strip()
            rating2 = get_safety_rating(d2)
            category2 = get_safety_category(d2)
            rationale2 = get_rationale(d2).strip()

            if image_id != image_id2:
                print('ID missmatch:', image_id, image_id2)

            # if check_repetition(rationale):
            if rating1 != rating2:
                print('Rating:', rating1, rating2)

            if category1 != category2:
                print('Category:', category1, category2)

            if rationale1 != rationale2:
                print('Rationale1:', rationale1)
                print('Rationale2:', rationale2)


            if rating1 != rating2 or category1 != category2 or rationale1 != rationale2:
                print(image_id)
                display(Image.open(image))
                print('=' * 50)
                showed += 1

    # print(image_id)
    # print('Rating:', rating1)
    # print('Category:', category1)
    # print('Rationale:', rationale1)
    #
    # display(Image.open(image))
    # print('=' * 50)
    # showed += 1
    # input_path = os.path.join(model_path, 'inputs', image_id + '.json')
    # output_path = os.path.join(model_path, 'model_output', image_id + '.json')
    # print(output_path)
    # raise Exception
    # if os.path.isfile(output_path):
    #     print('aaa')
    #     with open(output_path) as f, open(input_path) as f2:
    #         pred = json.load(f)
    #         pred = pred if isinstance(pred, dict) else json.loads(pred)
    #         input = json.load(f2)
    #         gt_ass = get_gt_assessment(d)
    #         if pred['category'] != gt_ass['category']:
    #             inconsistent_Category += 1
    #         else:
    #             cat_match += 1
    #         showed += 1
    #         print(image_id)
    #         print('Rating:', pred['rating'])
    #         print('Category:', pred['category'], get_safety_category(d))
    #         print('Rationale:', pred['rationale'])
    #         display(Image.open(image))
    #         print('=' * 50)
                    # if pred['rationale'] == '':
        #     # delete all invalid predictions
        #     os.remove(output_path)
        #     print('Deleted invalid predictions:', output_path)

    if showed > 20:
        break
        # delete all invalid predictions
print('Total:', len(outputs))
print('inconsistent_Category:', inconsistent_Category)
print('cat_match:', cat_match)

In [ ]:
# check quality of the model assessments
ds_path = '{local_data_dir}avaGuard-DS/json-v22-orig/eval.json'
model_pred_path = '{local_data_dir}avaGuard-v1.2-34b/LlavaGuard-DS/json-v22/sglang-bi-LlavaGuard-DS-json-v22/model_output'

from PIL import Image

data = json.load(open(ds_path))
data2 = json.load(open(ds_path2))


In [ ]:
# check quality of the model assessments
j = 0
TP, FP, TN, FN = 0, 0, 0, 0
invalid = 0
showed = 0
for sample in data[:]:
    #
    #     break
    id = sample['id']
    image = sample['image']
    safety_rating = get_safety_rating(sample)
    # if safety_rating or '_v3' in id:
    #     continue
    output_path = os.path.join(model_pred_path, id + '.json')
    if os.path.exists(output_path):
        with open(output_path) as f:
            try:
                pred = json.load(f)
                pred = pred if isinstance(pred, dict) else json.loads(pred)
                pred = pred['prediction']
                rating = pred['rating']
                category = pred['category']
                rationale = pred['rationale']
                if rating == 'Unsafe' and safety_rating == 'Unsafe':
                    TP += 1
                elif rating == 'Unsafe' and safety_rating == 'Safe':
                    FP += 1
                elif rating == 'Safe' and safety_rating == 'Safe':
                    TN += 1
                elif rating == 'Safe' and safety_rating == 'Unsafe':
                    FN += 1
                if check_repetition(pred['rationale']):
                    j += 1
                if rating != safety_rating and rating == 'Unsafe':
                    if showed < 20:
                        display(Image.open(image))
                        print(output_path)
                        print(f'Rating: {rating} (GT: {safety_rating})')
                        print(f'Category: {category} (GT: {get_safety_category(sample)})')
                        print(f'Rationale: {rationale}')
                        print('=' * 50)
                        showed += 1
            except:
                invalid += 1

print(j)
print('TP:', TP)
print('FP:', FP)
print('TN:', TN)
print('FN:', FN)
print('Total:', len(data))
print('Precision:', TP / (TP + FP))
print('Recall:', TP / (TP + FN))

In [ ]:
p = '{local_data_dir}tionales/llava-v1.6-34b-json-v16/model_output/000a2b54275771568ba2ddbe4a6e.json'
with open(p) as f:
    pred = json.load(f)
    print(pred['rationale'])
    print(type(pred))

In [ ]:
import glob
import json
def sort_model_paths(paths):
    # Helper to extract size in billions
    def extract_size(path):
        size_str = path.split('/')[-4].split('-')[2].lower().replace('b', '')
        return float(size_str)
    
    # Group by model family
    families = {}
    for path in paths:
        model_name = path.split('/')[-4].split('-')[0]
        if model_name not in families:
            families[model_name] = []
        families[model_name].append(path)
    
    # Sort each family by size
    sorted_paths = []
    for family in sorted(families.keys()):
        family_paths = families[family]
        sorted_paths.extend(sorted(family_paths, key=extract_size))
    
    return sorted_paths

# Usage:

paths = glob.glob('/pfss/mlde/workspaces/mlde_wsp_KIServiceCenter/lhelff/ds/LlavaGuard/eval/*/LlavaGuard-DS-v24/test_metrics.json')
paths += glob.glob('/pfss/mlde/workspaces/mlde_wsp_KIServiceCenter/lhelff/ds/LlavaGuard/eval/*/test_metrics.json')
paths = sorted(paths)
print(paths)
for p in paths:
    with open(p) as f:
        data = json.load(f)
        model_name = p.split('eval/')[1].split('/')[0]
        # print(model_name)
        model_name = model_name.replace('-MPO', '').replace('_', '.')
        model_name = model_name.replace('-Instruct', '').replace('-Vision', '')
        model_name = model_name.replace('-qwen2', '').replace('-ov', '').replace('-onevision', '-OV').replace('b-', 'B-')
        txt = f'{model_name} & \\cmark & \\cmark & \\cmark & {data["Balanced Accuracy"]*100:.2f} & {data["Recall"]*100:.2f} & {data["Specificity"]*100:.2f} & {data["Precision"]*100:.2f} & {data["F1"]*100:.2f}'
        if 'Inference Speed' in data:
            txt += f'& {data["Inference Speed"].split(" ")[0]} \\\\'
        else:
            txt += f'\\\\'
        print(txt)